Load the dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/extended_programming_code_snippets.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df

,Query,Code_Snippet,Language,Tags
0,How to create a list comprehension in Python?,[x**2 for x in range(10)],Python,tutorial
1,How to handle missing data in pandas?,"df.fillna(0, inplace=True)",Python,example
2,How to use a lambda function in Python?,lambda x: x + 2,Python,advanced
3,How to create a REST API in Flask?,from flask import Flask\napp = Flask(__name__)...,Python,tutorial
4,How to perform matrix multiplication in numpy?,"import numpy as np\nnp.dot(A, B)",Python,advanced
...,...,...,...,...
3015,How to implement a class in C++?,class MyClass {\npublic:\n void myMethod() ...,C++,tutorial
3016,How to use pointers in C++?,int x = 10;\nint* ptr = &x;,C++,advanced
3017,How to read a file in C++?,#include <fstream>\nstd::ifstream file('file.t...,C++,common-issues
3018,How to create a vector in C++?,"#include <vector>\nstd::vector<int> v = {1, 2,...",C++,tutorial


In [ ]:
df['Query']=df['Query'].str.lower()
df['Code_Snippet']=df['Code_Snippet'].str.lower()

In [ ]:
df

,Query,Code_Snippet,Language,Tags
0,how to create a list comprehension in python?,[x**2 for x in range(10)],Python,tutorial
1,how to handle missing data in pandas?,"df.fillna(0, inplace=true)",Python,example
2,how to use a lambda function in python?,lambda x: x + 2,Python,advanced
3,how to create a rest api in flask?,from flask import flask\napp = flask(__name__)...,Python,tutorial
4,how to perform matrix multiplication in numpy?,"import numpy as np\nnp.dot(a, b)",Python,advanced
...,...,...,...,...
3015,how to implement a class in c++?,class myclass {\npublic:\n void mymethod() ...,C++,tutorial
3016,how to use pointers in c++?,int x = 10;\nint* ptr = &x;,C++,advanced
3017,how to read a file in c++?,#include <fstream>\nstd::ifstream file('file.t...,C++,common-issues
3018,how to create a vector in c++?,"#include <vector>\nstd::vector<int> v = {1, 2,...",C++,tutorial


Tokenization

For text Generation always use T5 model

In [ ]:
from transformers import T5Tokenizer
#load t5 model
tokenizer=T5Tokenizer.from_pretrained('t5-small')
#Tokenize the dataset
def preprocess(df):
  inputs=['generate code:'+query for query in df['Query']]
  targets=df['Code_Snippet'].tolist()
  input_encodings=tokenizer(inputs,truncation=True,padding='max_length',max_length=128,return_tensors='pt')
  target_encodings=tokenizer(targets,truncation=True,padding='max_length',max_length=128,return_tensors='pt')
  return input_encodings,target_encodings

input_encodings,target_encodings=preprocess(df)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
input_encodings

{'input_ids': tensor([[3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0],
        ...,
        [3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0],
        [3806, 1081,   10,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Dataset preparation

In [ ]:
import torch

class CodeSnippetDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

# Wrap encodings into the dataset object
dataset = CodeSnippetDataset(input_encodings, target_encodings)





Fine tune the Model

In [ ]:
from transformers import T5ForConditionalGeneration,Trainer,TrainingArguments
# Check if CUDA (GPU) is available
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load T5 model and move it to the device (GPU or CPU)
model=T5ForConditionalGeneration.from_pretrained('t5-small')

# Define training arguments with optimized settings
training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    eval_strategy="epoch",  # Evaluation after each epoch
    save_strategy="epoch",        # Save after each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Reduce batch size to 4
    per_device_eval_batch_size=4,   # Reduce batch size to 4
    num_train_epochs=5,            # Train for fewer epochs (for testing)
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,             # Log less frequently
    save_total_limit=1,            # Limit the number of saved models
    load_best_model_at_end=True,
    fp16=True,                     # Enable mixed precision to speed up training
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

# Train the model
trainer.train()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vishalvishalmeena18 (vishalvishalmeena18-indian-institute-of-technology-kharagpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.161300,0.065865
2,0.079000,0.012349
3,0.047000,0.009092
4,0.026600,0.008612
5,0.024100,0.008483


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3775, training_loss=0.2096340591228561, metrics={'train_runtime': 495.0859, 'train_samples_per_second': 30.5, 'train_steps_per_second': 7.625, 'total_flos': 510915300556800.0, 'train_loss': 0.2096340591228561, 'epoch': 5.0})

Save fine tune model

In [ ]:
model.save_pretrained('/content/drive/MyDrive//content/t5_finetuned_model')
tokenizer.save_pretrained('/content/drive/MyDrive//content/t5_finetuned_model')

('/content/drive/MyDrive//content/t5_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive//content/t5_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive//content/t5_finetuned_model/spiece.model',
 '/content/drive/MyDrive//content/t5_finetuned_model/added_tokens.json')

Searh(Code Snippet Generation System)

In [ ]:
def generate_code(query):
 query= query.lower()
 input_ids=tokenizer.encode('generate code:'+query,return_tensors='pt',truncation=True,max_length=128).to(device)
 output=model.generate(input_ids,max_length=128,num_beams=4,early_stopping=True)
 generated_code=tokenizer.decode(output[0],skip_special_tokens=True)
 return generated_code

generate_code('Can you create a list in python ?')

'[x**2 for x in range(10)]'

In [ ]:
!pip install datasets


Fetching rows 0 to 100...
Fetching rows 100 to 200...
Fetching rows 200 to 300...
Fetching rows 300 to 400...
Fetching rows 400 to 500...
Fetching rows 500 to 600...
Fetching rows 600 to 700...
Fetching rows 700 to 800...
Fetching rows 800 to 900...
Fetching rows 900 to 1000...
Fetching rows 1000 to 1100...
Fetching rows 1100 to 1200...
Fetching rows 1200 to 1300...
Fetching rows 1300 to 1400...
Fetching rows 1400 to 1500...
Fetching rows 1500 to 1600...
Fetching rows 1600 to 1700...
Fetching rows 1700 to 1800...
Fetching rows 1800 to 1900...
Fetching rows 1900 to 2000...
Fetching rows 2000 to 2100...
Fetching rows 2100 to 2200...
Fetching rows 2200 to 2300...
Fetching rows 2300 to 2400...
Fetching rows 2400 to 2500...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>